In [258]:
#coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import random 

In [259]:
# 1. 数据预处理

def getTrainSet():
    
    #1-读数据存入list中
    df = pd.read_excel('Iris.xlsx',encoding ="utf-8")
    sl=df['sepal_length'].values.tolist()
    sw=df['sepal_width'].values.tolist()
    fl=df['flower_length'].values.tolist()
    fw=df['flower_width'].values.tolist()
    c=df['classify'].values.tolist()
    
    ori_set = []
    for a,b,c,d,e in zip(sl,sw,fl,fw,c):
        tmp = [a,b,c,d,e]
        ori_set.append(tmp)
    ori_set = np.array(ori_set)
    random.shuffle(ori_set) 
    
    #2-留出法分割训练数据与测试数据
    train = ori_set[:120     ,:ori_set.shape[1]-1]
    labels = ori_set[:120    ,ori_set.shape[1]-1]
    testData = ori_set[120:  ,:ori_set.shape[1]-1]
    testLabels = ori_set[12:,ori_set.shape[1]-1]

    return train, labels, testData, testLabels

train, labels, testData, testLabels = getTrainSet()

In [260]:
def predict(train, labels, each):
    
    # 先验概率
    P_c = Counter(labels)
    name_list = ['Iris-setosa','Iris-versicolor','Iris-virginica']
    for i in range(3):
        P_c[name_list[i]] = P_c.get(name_list[i]) /float(len(labels))

    #条件概率
    P_xc = {}
    P = {}
    for y in P_c.keys():
        yi = [i for i, label in enumerate(labels) if label == y]
        for j in range(len(each)):
            xi = [i for i, e in enumerate(train[:,j]) if e == each[j]]
            index = str(each[j]) + ':' + str(y)
            P_xc[index] = len(set(xi) & set(yi)) / len(labels)
        for x in each:
            index = str(x) + '|' + str(y)
            P[index] = P_xc[str(x) + ':' + str(y)] / float(P_c[y])

    #计算所有标签的可能概率，将最大概率对象返回作为预测结果
    prob_list = P_c.copy()
    for y in P_c:
        for x in each:
            prob_list[y] *= P[str(x)+'|'+str(y)]

    return max(prob_list, key = prob_list.get)

In [261]:
# 3.测试：

def cal_accuracy(train, labels, testData, testLabels):
    correct = 0
    for each,eva in zip(testData,testLabels):
        predict_result = predict(train, labels, each)             #传入测试
        
        #正误判断
        if predict_result == eva:
            correct += 1
    accuracy = correct/float(len(testData))
    print('accuracy : ',accuracy * 100, '%')


In [262]:
cal_accuracy(train, labels, testData, testLabels)

accuracy :  83.33333333333334 %
